# 概述

这本笔记本是基于这个[预测和特征存储在线服务](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/mobile_gaming/mobile_gaming_feature_store.ipynb)的笔记本和[这篇博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。

### 数据集

这个数据集是来自一个名为"Flood It!"（Android，iOS）的实际移动游戏应用的公共样本导出数据。

### 模型

您在这个笔记本中使用的模型基于[这篇博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。这个模型的理念是，您的公司有大量的日志数据，描述了游戏用户与网站的交互方式。原始数据包含以下信息类别：

- 身份 - 唯一的玩家身份编号
- 人口统计特征 - 关于玩家的信息，比如玩家所在的地理区域
- 行为特征 - 玩家触发某些游戏事件的次数，比如达到新的等级
- 流失倾向 - 这是标签或目标特征，它提供了该玩家可能流失的概率，即停止成为活跃玩家。

上面引用的博客文章解释了如何使用BigQuery存储原始数据，预处理数据用于机器学习，以及训练相应的模型。因为这个笔记本专注于模型监控，而不是训练模型，您将重复使用一个预先训练好的版本该模型，它已经被导出到了云存储中，存储在`gs://mco-mm/churn`中。

基本设置

安装包

In [ ]:
! pip install google-cloud-aiplatform

## 创建项目

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

In [ ]:
! gcloud config set project $PROJECT_ID

In [ ]:
REGION = ""

if REGION == "" or REGION is None:
    REGION = "us-central1"

时间戳

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

请验证您的谷歌云账户

如果您正在使用Vertex AI工作台笔记本，则您的环境已经经过身份验证。跳过这一步。

如果您正在使用Colab，请运行下面的单元格，并按提示进行操作以通过oAuth验证您的帐户。

否则，请按照以下步骤操作：

1. 在Cloud控制台中，转到创建服务帐户密钥页面。

2. 点击创建服务帐户。

3. 在服务帐户名称字段中输入名称，然后点击创建。

4. 在授予此服务帐户访问项目部分，单击角色下拉列表并添加以下角色：

- BigQuery管理员
- 存储管理员
- 存储对象管理员
- Vertex AI管理员
- Vertex AI特征存储管理员

5. 点击创建。一个包含您的密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中将您的服务帐户密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个存储桶，并将导出的模型复制到其中。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

In [ ]:
! gsutil ls -al $BUCKET_URI

将训练好的模型复制到你的存储桶中。

In [ ]:
! gsutil cp -r gs://mco-mm/churn/* $BUCKET_URI

### 创建一个服务账号

我们需要为这个新功能创建一个服务账号，因为预测工作负载的凭据没有访问特征存储库的权限。创建一个账号并授予`roles/aiplatform.serviceAgent`角色，这将授予它访问大多数预测资源和特征存储库的权限。

In [ ]:
SA_NAME = "prediction-and-fs-testing"
SA_DESCRIPTION = '"SA to test Prediction and Feature Store integration"'
DISPLAY_NAME = "prediction-and-fs-testing"

In [ ]:
!gcloud iam service-accounts create $SA_NAME \
    --description=$SA_DESCRIPTION \
    --display-name=$DISPLAY_NAME

In [ ]:
# SERVICE_ACCOUNT = "prediction-and-fs@bon-test-0.iam.gserviceaccount.com"
SERVICE_ACCOUNT = (
    f"{SA_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"  # @param {type:"string"}
)

赋予服务账户`Storage Admin`和`Vertex AI Feature Store Data Viewer`角色，以便访问GCS中的工件和Feature Store中的数据。

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/storage.objectAdmin;

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/aiplatform.featurestoreDataViewer;

## 导入库

In [ ]:
# General
import os
import sys

# Vertex AI and its Feature Store
from google.cloud import aiplatform
from google.cloud.aiplatform import Featurestore

注意：该功能现在仅在“autopush”中可用。

In [ ]:
aiplatform.constants.base.API_BASE_PATH = "autopush-aiplatform.sandbox.googleapis.com"
aiplatform.constants.base.PREDICTION_API_BASE_PATH = (
    "autopush-prediction-aiplatform.sandbox.googleapis.com"
)

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

创建特征存储#

In [ ]:
FEATURESTORE_ID = "mobile_gaming_" + TIMESTAMP  # @param {type:"string"}

# Vertex AI Feature store
ONLINE_STORE_NODES_COUNT = 5
DEMOGRAPHIC_ENTITY_ID = "demographic"
BEHAVIOR_ENTITY_ID = "behavior"
FEATURE_TIME = "timestamp"
ENTITY_ID_FIELD = "user_pseudo_id"

我们将使用此[博客帖子](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)中使用的数据集的导出样本数据。

In [ ]:
SOURCE_URI = "gs://featurestore_prediction_integration/data/mobile_gaming_dataset.csv"

## 创建特征库

In [ ]:
try:
    mobile_gaming_feature_store = Featurestore.create(
        featurestore_id=FEATURESTORE_ID,
        online_store_fixed_node_count=ONLINE_STORE_NODES_COUNT,
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    FEATURESTORE_RESOURCE_NAME = mobile_gaming_feature_store.resource_name
    print(f"Feature store created: {FEATURESTORE_RESOURCE_NAME}")

创建实体

In [ ]:
try:
    demographic_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=DEMOGRAPHIC_ENTITY_ID,
        description="User demographic Entity",
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    DEMOGRAPHIC_ENTITY_RESOURCE_NAME = demographic_entity_type.resource_name
    print("Entity type name is", DEMOGRAPHIC_ENTITY_RESOURCE_NAME)

In [ ]:
try:
    behavior_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=BEHAVIOR_ENTITY_ID, description="User behavior Entity", sync=True
    )
except RuntimeError as error:
    print(error)
else:
    BEHAVIOR_ENTITY_RESOURCE_NAME = behavior_entity_type.resource_name
    print("Entity type name is", BEHAVIOR_ENTITY_RESOURCE_NAME)

### 创建特征

#### 特性配置

In [ ]:
demographic_feature_configs = {
    "country": {
        "value_type": "STRING",
        "description": "The country of customer",
        "labels": {"status": "passed"},
    },
    "operating_system": {
        "value_type": "STRING",
        "description": "The operating system of device",
        "labels": {"status": "passed"},
    },
    "language": {
        "value_type": "STRING",
        "description": "The language of device",
        "labels": {"status": "passed"},
    },
    "user_pseudo_id": {
        "value_type": "STRING",
        "description": "User pseudo id",
        "labels": {"status": "passed"},
    },
}

behavior_feature_configs = {
    "cnt_user_engagement": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement level",
        "labels": {"status": "passed"},
    },
    "cnt_level_start_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with start level",
        "labels": {"status": "passed"},
    },
    "cnt_level_end_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with end level",
        "labels": {"status": "passed"},
    },
    "cnt_level_complete_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with complete status",
        "labels": {"status": "passed"},
    },
    "cnt_level_reset_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with reset status",
        "labels": {"status": "passed"},
    },
    "cnt_post_score": {
        "value_type": "DOUBLE",
        "description": "A variable of user score",
        "labels": {"status": "passed"},
    },
    "cnt_spend_virtual_currency": {
        "value_type": "DOUBLE",
        "description": "A variable of user virtual amount",
        "labels": {"status": "passed"},
    },
    "cnt_ad_reward": {
        "value_type": "DOUBLE",
        "description": "A variable of user reward",
        "labels": {"status": "passed"},
    },
    "cnt_challenge_a_friend": {
        "value_type": "DOUBLE",
        "description": "A variable of user challenges with friends",
        "labels": {"status": "passed"},
    },
    "cnt_completed_5_levels": {
        "value_type": "DOUBLE",
        "description": "A variable of user level 5 completed",
        "labels": {"status": "passed"},
    },
    "cnt_use_extra_steps": {
        "value_type": "DOUBLE",
        "description": "A variable of user extra steps",
        "labels": {"status": "passed"},
    },
    "month": {
        "value_type": "INT64",
        "description": "First touch month",
        "labels": {"status": "passed"},
    },
    "julianday": {
        "value_type": "INT64",
        "description": "First touch julian day",
        "labels": {"status": "passed"},
    },
    "dayofweek": {
        "value_type": "INT64",
        "description": "First touch day of week",
        "labels": {"status": "passed"},
    },
}

使用`batch_create_features`方法创建特性。

In [ ]:
try:
    demographic_entity_type.batch_create_features(
        feature_configs=demographic_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in demographic_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

In [ ]:
try:
    behavior_entity_type.batch_create_features(
        feature_configs=behavior_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in behavior_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

摄取特性

In [ ]:
DEMOGRAPHIC_FEATURES_IDS = [
    feature.name for feature in demographic_entity_type.list_features()
]

In [ ]:
try:
    demographic_entity_type.ingest_from_gcs(
        feature_ids=DEMOGRAPHIC_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        gcs_source_uris=SOURCE_URI,
        gcs_source_type="csv",
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=True,
    )
except RuntimeError as error:
    print(error)

In [ ]:
BEHAVIOR_FEATURES_IDS = [
    feature.name for feature in behavior_entity_type.list_features()
]

In [ ]:
try:
    behavior_entity_type.ingest_from_gcs(
        feature_ids=BEHAVIOR_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        gcs_source_uris=SOURCE_URI,
        gcs_source_type="csv",
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=True,
    )
except RuntimeError as error:
    print(error)

创造要素获取配置

## 特征提取配置协议：

```protobuf
message FeatureFetchConfig {
  // 在特征被提取后，内部预测请求的格式会自动生成。目前，预测支持XGBoost、TensorFlow和scikit-learn，即将开始支持Pytorch。在这些框架中，XGBoost仅支持数组输入格式（即输入特征以数组形式呈现），而其他三个框架则可以允许字典格式和数组格式的输入。
  ModelInputFormat model_input_format = 3;
  enum ModelInputFormat {
    MODEL_INPUT_FORMAT_UNSPECIFIED = 0;
    ARRAY = 1;
    DICT = 2;
  }

  // 指定预测输入的细节
  repeated Feature features = 4;
  message Feature {
    // 在internal_request_format = DICT时，这个value_key被用作内部预测请求中特征值的键。在FeatureFetchConfig中，通过只使用value_key的Feature消息来表示透传特征。
    string value_key = 1;

    // 定义每个特征来自哪个特征存储。
    FeatureSource feature_source = 2;
    message FeatureSource {
      // 从高层来看，FeatureSource有两个字段，entity_id_key是关于从哪里提取值的“行”，feature_resource_path是关于从哪里提取值的“列”。

      // 指定用户发送给预测服务的外部请求中包含实体ID的键。
      string entity_id_key = 1;

      // URL格式的资源路径，用于识别实体类型。
      // 格式应为
      // projects/PROJECT/locations/LOCATION/featurestores/FEATURESTORE_ID/entityTypes/ENTITY_TYPE_ID/
      // 例如
      // "projects/my-feature-store-project/locations/us-central1/featurestores/movie_predictions/entityTypes/movies/"
      string entity_type = 2;

      // 在特征存储中定义的特征ID。
      string feature_id = 3;
    }
  }
}
```

生成特性提取配置

In [ ]:
FEATURE_FETCH_CONFIG_TEMPLATE = """modelInputFormat: DICT
features:
- valueKey: user_pseudo_id
- valueKey: country
  featureSource:
    entityIdKey: demographic
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    featureId: country
- valueKey: operating_system
  featureSource:
    entityIdKey: demographic
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    featureId: operating_system
- valueKey: language
  featureSource:
    entityIdKey: demographic
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    featureId: language
- valueKey: cnt_user_engagement
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_user_engagement
- valueKey: cnt_level_start_quickplay
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_level_start_quickplay
- valueKey: cnt_level_end_quickplay
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_level_end_quickplay
- valueKey: cnt_level_complete_quickplay
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_level_complete_quickplay
- valueKey: cnt_level_reset_quickplay
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_level_reset_quickplay
- valueKey: cnt_post_score
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_post_score
- valueKey: cnt_spend_virtual_currency
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_spend_virtual_currency
- valueKey: cnt_ad_reward
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_ad_reward
- valueKey: cnt_challenge_a_friend
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_challenge_a_friend
- valueKey: cnt_completed_5_levels
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_completed_5_levels
- valueKey: cnt_use_extra_steps
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: cnt_use_extra_steps
- valueKey: month
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: month
- valueKey: julianday
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: julianday
- valueKey: dayofweek
  featureSource:
    entityIdKey: behavior
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    featureId: dayofweek"""

In [ ]:
feature_fetch_config = FEATURE_FETCH_CONFIG_TEMPLATE.format(
    PROJECT_NUMBER=PROJECT_NUMBER, REGION=REGION, FEATURESTORE_ID=FEATURESTORE_ID
)

In [ ]:
print(feature_fetch_config)

In [ ]:
with open("prediction_featuresstore_fetch_config.yaml", "w") as f:
    f.write(feature_fetch_config)

In [ ]:
# Remove if the file already exists
!gsutil rm $BUCKET_URI/prediction_featuresstore_fetch_config.yaml

In [ ]:
!gsutil cp prediction_featuresstore_fetch_config.yaml $BUCKET_URI

与顶点预测集成

上传模型

In [ ]:
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"
DISPLAY_NAME = "mobile_gaming_featureStore_integration_"

In [ ]:
model = aiplatform.Model.upload(
    display_name=DISPLAY_NAME + TIMESTAMP,
    artifact_uri=BUCKET_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

## 在线预测

### 部署模型

In [ ]:
DEPLOYED_NAME = DISPLAY_NAME + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MACHINE_TYPE = "n1-standard-4"

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=MACHINE_TYPE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    service_account=SERVICE_ACCOUNT,
)

### 预测

In [ ]:
default_pred_request = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "country": "Australia",
        "operating_system": "IOS",
        "language": "en-au",
        "cnt_user_engagement": 3.0,
        "cnt_level_start_quickplay": 1.0,
        "cnt_level_end_quickplay": 0.0,
        "cnt_level_complete_quickplay": 0.0,
        "cnt_level_reset_quickplay": 0.0,
        "cnt_post_score": 0.0,
        "cnt_spend_virtual_currency": 0.0,
        "cnt_ad_reward": 0.0,
        "cnt_challenge_a_friend": 0.0,
        "cnt_completed_5_levels": 0.0,
        "cnt_use_extra_steps": 0.0,
        "month": 7,
        "julianday": 194,
        "dayofweek": 6,
    },
    {
        "user_pseudo_id": "E5D16173400729E05DFFB4883FA6EF1C",
        "country": "United States",
        "operating_system": "IOS",
        "language": "en-us",
        "cnt_user_engagement": 1.0,
        "cnt_level_start_quickplay": 1.0,
        "cnt_level_end_quickplay": 0.0,
        "cnt_level_complete_quickplay": 0.0,
        "cnt_level_reset_quickplay": 0.0,
        "cnt_post_score": 0.0,
        "cnt_spend_virtual_currency": 0.0,
        "cnt_ad_reward": 0.0,
        "cnt_challenge_a_friend": 0.0,
        "cnt_completed_5_levels": 0.0,
        "cnt_use_extra_steps": 0.0,
        "month": 6,
        "julianday": 173,
        "dayofweek": 6,
    },
]

In [ ]:
fs_pred_request = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "demographic": "AB0F2EE5F9F401763BE1E9FA55410312",
        "behavior": "AB0F2EE5F9F401763BE1E9FA55410312",
    },
    {
        "user_pseudo_id": "E5D16173400729E05DFFB4883FA6EF1C",
        "demographic": "E5D16173400729E05DFFB4883FA6EF1C",
        "behavior": "E5D16173400729E05DFFB4883FA6EF1C",
    },
]

从特征存储中提取的特征可以被覆盖。

In [ ]:
fs_pred_request_with_overridden_features = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "demographic": "AB0F2EE5F9F401763BE1E9FA55410312",
        "behavior": "AB0F2EE5F9F401763BE1E9FA55410312",
        "cnt_ad_reward": 10.0,
        "cnt_challenge_a_friend": 10.0,
        "cnt_completed_5_levels": 10.0,
        "cnt_use_extra_steps": 10.0,
    },
    {
        "user_pseudo_id": "E5D16173400729E05DFFB4883FA6EF1C",
        "demographic": "E5D16173400729E05DFFB4883FA6EF1C",
        "behavior": "E5D16173400729E05DFFB4883FA6EF1C",
        "cnt_ad_reward": 10.0,
        "cnt_challenge_a_friend": 10.0,
        "cnt_completed_5_levels": 10.0,
        "cnt_use_extra_steps": 10.0,
    },
]

In [ ]:
endpoint.predict([default_pred_request[0]])

In [ ]:
default_response = endpoint.predict(default_pred_request)

In [ ]:
print(default_response)

单例

In [ ]:
endpoint.predict([fs_pred_request[0]])

In [ ]:
endpoint.predict([fs_pred_request_with_overridden_features[0]])

多个实例

In [ ]:
fs_response = endpoint.predict(fs_pred_request)

In [ ]:
print(fs_response)

In [ ]:
fs_with_overridden_features_response = endpoint.predict(
    fs_pred_request_with_overridden_features
)

In [ ]:
print(fs_with_overridden_features_response)

### 比较响应

In [ ]:
print(default_response.predictions == fs_response.predictions)

# 清理

In [ ]:
# delete feature store
mobile_gaming_feature_store.delete(sync=True, force=True)

In [ ]:
# delete Vertex AI resources
endpoint.undeploy_all()
endpoint.delete()
model.delete

In [ ]:
# Delete bucket
!gsutil -m rm -r $BUCKET_URI